In [1]:
import qml
import numpy as np

import sys
sys.path.insert(0, '/home/misa/APDFT/prototyping/atomic_energies/')
import alchemy_tools as alch
import qml_interface as qi

In [2]:
def save(set_sizes, error, std, fname):
    big_array = np.array([set_sizes, error, std]).T
#    big_array = np.flip(big_array, axis=0)
    np.savetxt(fname, big_array, delimiter='\t', header='set_size\t error \t error_std')

In [8]:
# load data (with mic)
paths=qi.wrapper_alch_data()
alchemy_data, molecule_size = qi.load_alchemy_data(paths)

# prepare representations and labels
LABEL_VALUE = 'atomisation'
local_reps = qi.generate_atomic_representations(alchemy_data, molecule_size)
local_labels = qi.generate_label_vector(alchemy_data, molecule_size.sum(), value=LABEL_VALUE)

# # read optimized hyperparameters
# HYPAR_PATH = '/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/mic/optimized_hyperpar_atomic_mic.txt'
# with open(HYPAR_PATH, 'r') as f:
#     lines = f.readlines()
sigma = 432.8761281083057 #float(lines[2].split('\t')[0])
lam = 1e-7 #float(lines[2].split('\t')[1])


In [9]:
# define number of training points for which MAE is calculated
set_sizes = np.logspace(0,12, 13, base=2).astype(int)

# calculate error for every training point size
error_cv = np.zeros(len(set_sizes))
error_std = np.zeros(len(set_sizes))

In [10]:
for idx, tr_size in enumerate(set_sizes):
        error_cv[idx], error_std[idx] = qi.crossvalidate_per_atom(local_reps, local_labels, molecule_size, tr_size, sigma, lam, num_cross=10)

In [11]:
name = '/home/misa/APDFT/prototyping/atomic_energies/results/analyse_learning/mic/lcurve_atomisation_atomic_per_atom.txt'
save(set_sizes, error_cv, error_std, name)